### 安装 git lfs

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs && git lfs install

### 下载模型

In [ ]:
!git clone https://huggingface.co/THUDM/chatglm-6b ../chatglm-6b

### 安装依赖

In [ ]:
!pip install -r ./requirement.txt

### 数据集准备 

准备 `.jsonl` 格式的数据放到 ./data 目录下。数据格式为：


{"q": "问题", "a": "回答"}


### 对数据集进行分词

为了避免每次训练的时都要重新对数据集分词，先分好词形成特征后保存成可直接用于训练的数据集。相关参数说明：

* model_checkpoint: 模型目录
* input_file:  ./data 目录下的数据集文件名
* prompt_key:  数据集中 prompt 对应的字段（这里是 q）
* target_key:  数据集中 completion 对应的字段（这里是 a)
* save_name:  数据集保存目录，分词后的数据保存在 ./data/tokenized_data 下
* max_seq_length:  文本最大长度

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python ./script/tokenize_dataset_rows.py \
    --model_checkpoint ./chatglm-6b \
    --input_file dataset.jsonl \
    --prompt_key q \
    --target_key a \
    --save_name dataset \
    --max_seq_length 2000 \
    --skip_overlength False

### 使用 LoRA 微调

参数说明：

* tokenized_dataset: 分词后的数据集保存目录（即上一步 save_name 的值）
* tlora_rank: 设置 LoRA 的秩，推荐为4或8，显存够的话使用8
* tper_device_train_batch_size: 每块 GPU 上的 batch size
* tgradient_accumulation_steps: 梯度累加，可以在不提升显存占用的情况下增大 batch size
* tmax_steps: 训练步数
* tsave_steps: 多少步保存一次
* tsave_total_limit: 保存多少个checkpoint
* tlogging_steps: 多少步打印一次训练情况(loss, lr, etc.)
* toutput_dir: 模型文件保存地址

In [ ]:
# 删除上次的微调模型
# !rm -rf /mnt/workspace/glm-fine-tuning/weights

!CUDA_VISIBLE_DEVICES=0 python ./script/chatglm_lora_tuning.py \
    --tokenized_dataset dataset \
    --lora_rank 8 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --max_steps 5000 \
    --save_steps 200 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir ./weights/api-fn

### 加载微调模型

微调模型保存在上一步配置的 output_dir 目录下。至少需要其中的 adapter_model.bin、adapter_config.json 两个文件才能部署成功

### 启动 web 服务

In [ ]:
!python ./server/web.py

### 通过 API 服务测试模型

In [ ]:
# 安装 pyngrok 用来暴露服务

!pip install pyngrok

In [3]:
# 启动服务
from pyngrok import ngrok, conf

pyngrok_config = conf.PyngrokConfig(
    auth_token="2RjdBXCzDODfrigSoG9RaShlY4w_5mkv6Dp6Bh8yg4YtLXK4E",
)
conf.set_default(pyngrok_config)

ssh_tunnel = ngrok.connect(8000, "http")

# 打印代理地址
print("proxy url:", ssh_tunnel.public_url)

t=2023-07-07T16:08:26+0800 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml


proxy url: https://bfea-47-110-48-61.ngrok-free.app


In [ ]:
# 后台运行 chatlm
get_ipython().system_raw("python ./server/api.py &")